In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
async def get_expiry(f_today, index):

    if index == 'bnf':    
        if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
            f_expiry = dt.date(2024, 1, 25)
        elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
            f_expiry = dt.date(2024, 1, 31)
        elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
            f_expiry = dt.date(2024, 2, 29)
        elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
            f_expiry = dt.date(2024, 2, 27)
        elif f_today < dt.date(2023, 9, 1):
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
        elif f_today >= dt.date(2023, 9, 1):
            if f_today.day < 24:
                days_to_wednesday = (2 - f_today.weekday()) % 7
                nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
                f_expiry = nearest_wednesday
                if nse.valid_days(
                    start_date=nearest_wednesday, end_date=nearest_wednesday
                ).empty:
                    f_expiry = nearest_wednesday - dt.timedelta(days=1)
            else:
                days_to_thursday = (3 - f_today.weekday()) % 7
                nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
                f_expiry = nearest_thursday
                if nse.valid_days(
                    start_date=nearest_thursday, end_date=nearest_thursday
                ).empty:
                    f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'nifty':
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'finnifty' or index == 'fnf':
        days_to_thursday = (1 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'midcpnifty' or index == 'midcp':
        days_to_thursday = (0 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [5]:
bnf_1min = pd.read_csv("../data/gold_tv_4h.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["time"])
bnf_1min = bnf_1min[bnf_1min["datetime"].dt.year >= 2012]

In [6]:
bnf_1min.head()

,time,open,high,low,close,MA,Shapes,Shapes.1,RSI,datetime
0,2015-03-02T09:00:00+05:30,26600,26769,26600,26700,NaN,0,0,NaN,2015-03-02 09:00:00+05:30
1,2015-03-02T13:00:00+05:30,26709,26730,26653,26680,NaN,0,0,NaN,2015-03-02 13:00:00+05:30
2,2015-03-02T17:00:00+05:30,26684,26742,26576,26619,NaN,0,0,NaN,2015-03-02 17:00:00+05:30
3,2015-03-02T21:00:00+05:30,26623,26624,26500,26506,NaN,0,0,NaN,2015-03-02 21:00:00+05:30
4,2015-03-03T09:00:00+05:30,26500,26525,26444,26463,NaN,0,0,NaN,2015-03-03 09:00:00+05:30


In [7]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [8]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
list_of_traded_dates = set(bnf_1min["datetime"].dt.date)
list_of_traded_dates

{datetime.date(2015, 8, 20),
 datetime.date(2023, 7, 26),
 datetime.date(2018, 1, 15),
 datetime.date(2016, 6, 30),
 datetime.date(2018, 5, 21),
 datetime.date(2022, 4, 25),
 datetime.date(2018, 2, 7),
 datetime.date(2017, 2, 9),
 datetime.date(2023, 10, 26),
 datetime.date(2024, 9, 5),
 datetime.date(2016, 5, 31),
 datetime.date(2019, 1, 25),
 datetime.date(2017, 6, 14),
 datetime.date(2019, 1, 1),
 datetime.date(2019, 8, 8),
 datetime.date(2017, 1, 17),
 datetime.date(2019, 11, 26),
 datetime.date(2022, 11, 8),
 datetime.date(2021, 12, 30),
 datetime.date(2022, 3, 31),
 datetime.date(2019, 10, 3),
 datetime.date(2016, 10, 19),
 datetime.date(2022, 3, 28),
 datetime.date(2022, 5, 24),
 datetime.date(2015, 6, 15),
 datetime.date(2017, 2, 10),
 datetime.date(2023, 6, 20),
 datetime.date(2022, 5, 3),
 datetime.date(2018, 1, 12),
 datetime.date(2018, 4, 16),
 datetime.date(2016, 8, 12),
 datetime.date(2023, 3, 28),
 datetime.date(2015, 3, 6),
 datetime.date(2017, 2, 15),
 datetime.date(20

In [9]:
def rename_ohlc_columns(df: pl.DataFrame) -> pl.DataFrame:

    column_mapping = {"o": "open", "h": "high", "l": "low", "c": "close", "v": "volume"}
    df = df.rename(column_mapping)

    return df

In [10]:
PORTFOLIO_VALUE = 10_00_000 # 10 Lacs
RPT_PCT = 0.01 # 1% RPT
SLIPPAGE_ = 0.001
LEVERAGE_ = 5

In [18]:
def calculate_signals(df, ema_length, x_days, pct):
    # Calculate X-day High/Low and 20 EMA
    df['X_High'] = df['high'].rolling(x_days).mean()
    df['X_Low'] = df['low'].rolling(x_days).mean()
    df['EMA_20'] = df['close'].ewm(span=ema_length, adjust=False).mean()
    
    # Trend conditions
    # df['EMA_Trend'] = df['EMA_20'] > df['EMA_20'].shift(1)
    
    # Signal conditions
    # df['Buy_Signal'] = (
    #     (df['close'] > df['X_High'].shift(1) * (1 + (pct / 100))) & 
    #     (df['EMA_Trend']) & 
    #     (df['close'] > df['EMA_20'])
    # )
    # df['Sell_Signal'] = (
    #     (df['close'] < df['X_Low'].shift(1) * (1 - (pct / 100))) & 
    #     (~df['EMA_Trend']) & 
    #     (df['close'] < df['EMA_20'])
    # )
    
    df['MA'] = df['close'].rolling(9).mean()
    df['EMA'] = df['close'].ewm(span=30, adjust=False).mean()

    df['Buy_Signal'] = (
       (df['MA'] > df['EMA']) &
        (df['MA'].shift(1) <= df['EMA'].shift(1))
    )
    return df

In [19]:
# def backtest(df):
#     position = 0
#     entry_price = 0
#     trades = []
#     trailing_stop = None
    
#     for i in range(len(df)):
#         if df.loc[i, 'Buy_Signal'] and position == 0:
#             position = 1
#             # entry_price = df.loc[i, 'close']
#             trailing_stop_long = df.loc[i, 'EMA_20']
#             # trades.append({'Type': 'Buy', 'Price': entry_price, 'Date': df.loc[i, 'datetime']})
#             entry_time_long = df.loc[i, 'datetime']
#             entry_price_long = df.loc[i, 'close']
#             initial_sl_long = trailing_stop_long
        
#         elif df.loc[i, 'Sell_Signal'] and position == 0:
#             position = -1
#             entry_price = df.loc[i, 'close']
#             trailing_stop_short = df.loc[i, 'EMA_20']
#             # trades.append({'Type': 'Sell', 'Price': entry_price, 'Date': df.loc[i, 'datetime']})
#             entry_time_short = df.loc[i, 'datetime']
#             entry_price_short = df.loc[i, 'close']
#             initial_sl_short = trailing_stop_short
        
#         elif position == 1:
#             if df.loc[i, 'low'] < trailing_stop_long:
#                 # trades.append({'Type': 'Exit', 'Price': df.loc[i, 'close'], 'Date': df.loc[i, 'datetime']})
#                 position = 0
#                 exit_time_long = df.loc[i, 'datetime']
#                 exit_price_long = trailing_stop_long
#                 points_long = exit_price_long - entry_price_long
                
#             elif df.loc[i, 'Add_Long']:
#                 # trades.append({'Type': 'Add Long', 'Price': df.loc[i, 'close'], 'Date': df.loc[i, 'datetime']})
#                 entry_time_long = df.loc[i, 'datetime']
#                 entry_price_long = df.loc[i, 'close']
#                 initial_sl_long = trailing_stop_long
#                 trailing_stop_long = max(trailing_stop_long, df.loc[i, 'EMA_20'])
        
#         elif position == -1:
#             if df.loc[i, 'high'] > trailing_stop:
#                 trades.append({'Type': 'Exit', 'Price': df.loc[i, 'close'], 'Date': df.loc[i, 'datetime']})
#                 position = 0
#             elif df.loc[i, 'Add_Short']:
#                 trades.append({'Type': 'Add Short', 'Price': df.loc[i, 'close'], 'Date': df.loc[i, 'datetime']})
#                 trailing_stop = min(trailing_stop, df.loc[i, 'EMA_20'])
    
#     return pd.DataFrame(trades)

In [57]:
def backtest(df, y_days):
    # Variables for long trades
    long_position = 0  # 0 = no position, 1 = long
    long_entry_price = 0
    long_entry_date = None
    long_trades = []
    tradebook = pd.DataFrame()
    tradebook_long = pd.DataFrame()
    tradebook_short = pd.DataFrame()
    long_trailing_stop = None
    # Variables for short trades
    short_position = 0  # 0 = no position, -1 = short
    short_entry_price = 0
    short_entry_date = None
    short_trades = []
    short_trailing_stop = None

    can_add_long = False
    can_add_short = False

    trade_number = 0
    lock_initial_sl = False
    first_sl = 0

    lock_initial_sl_2 = False
    first_sl_2 = 0

    for i in range(len(df)):
        # Entry signals for long trades
        long_trailing_stop = df.loc[i, 'EMA_20']
        # short_trailing_stop = df.loc[i, 'EMA_20']
        
        if df.loc[i, 'Buy_Signal'] and long_position == 0:
            long_position = 1
            long_entry_price = df.loc[i, 'close']
            long_entry_date = df.loc[i, 'datetime']
            # long_trailing_stop = df.loc[i, 'EMA_20']
            long_initial_sl = long_entry_price * 0.985

            tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
                'Trade No.': trade_number,
                'Entry_Date': df.loc[i, 'datetime'],
                'Exit_Date': None,
                'Trade_Type': 'Long',
                'Entry_Price': df.loc[i, 'close'],
                'Initial SL': long_initial_sl,
                'Exit_Price': None,
                # 'Profit/Loss': None  # Filled when exited
            }])], ignore_index=True)

            if not lock_initial_sl:
                first_sl = long_initial_sl
                lock_initial_sl = True
                
        # Exit or add to long position
        if long_position == 1:
            
            y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
            y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
            # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
            if df.loc[i, 'low'] <= long_initial_sl:
                tradebook_long.loc[
                    (tradebook_long['Trade No.'] == trade_number),
                    ['Exit_Date', 'Exit_Price', 'Exit Remark']
                ] = [df.loc[i, 'datetime'] , long_initial_sl , 'Initial SL Hit']
                long_position = 0
                can_add_long = False
                trade_number += 1
                lock_initial_sl = False
                first_sl = 0

            elif df.loc[i, 'MA'] <= df.loc[i, 'EMA']:
                tradebook_long.loc[
                    (tradebook_long['Trade No.'] == trade_number),
                    ['Exit_Date', 'Exit_Price', 'Exit Remark']
                ] = [df.loc[i, 'datetime'] , df.loc[i, 'close'] , 'MA Cross Reversal']
                long_position = 0
                can_add_long = False
                trade_number += 1
                lock_initial_sl = False
                first_sl = 0
            
            # elif df.loc[i, 'close'] <= long_trailing_stop:
            #     tradebook_long.loc[
            #         (tradebook_long['Trade No.'] == trade_number),
            #         ['Exit_Date', 'Exit_Price', 'Exit Remark']
            #     ] = [df.loc[i, 'datetime'] , df.loc[i, 'close'] , 'Trailing SL Hit']
            #     long_position = 0
            #     can_add_long = False
            #     trade_number += 1
            #     lock_initial_sl = False
            #     first_sl = 0

            elif (df.loc[i, 'low'] <= y_days_low) and not can_add_long:
                # Previous Y Candle Low Breached, confirmation found. Add position on high break
                can_add_long = True

            if can_add_long and long_position:
                if df.loc[i, 'high'] >= y_days_high:
                    tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
                        'Trade No.': trade_number,
                        'Entry_Date': df.loc[i, 'datetime'],
                        'Exit_Date': None,
                        'Trade_Type': 'Add_Long',
                        'Entry_Price': y_days_high,
                        'Initial SL': first_sl,
                        'Exit_Price': None,
                        # 'Profit/Loss': None  # Filled when exited
                    }])], ignore_index=True)
                    can_add_long = False

    # for i in range(len(df)):
        
    #     # Entry signals for long trades
    #     long_trailing_stop = df.loc[i, 'EMA_20']
    #     short_trailing_stop = df.loc[i, 'EMA_20']

    #     # Entry signals for short trades
    #     if df.loc[i, 'Sell_Signal'] and short_position == 0:
    #         short_position = -1
    #         short_entry_price = df.loc[i, 'close']
    #         short_entry_date = df.loc[i, 'datetime']
    #         short_trailing_stop = df.loc[i, 'EMA_20']
    #         short_initial_sl = df.loc[i, 'X_High']

    #         tradebook_short = pd.concat([tradebook_short, pd.DataFrame([{
    #             'Trade No.': trade_number,
    #             'Entry_Date': df.loc[i, 'datetime'],
    #             'Exit_Date': None,
    #             'Trade_Type': 'Short',
    #             'Entry_Price': df.loc[i, 'close'],
    #             'Initial SL': short_initial_sl,
    #             'Exit_Price': None,
    #             # 'Profit/Loss': None  # Filled when exited
    #         }])], ignore_index=True)
            
    #         if not lock_initial_sl_2:
    #             first_sl_2 = short_initial_sl
    #             lock_initial_sl_2 = True

    #     if short_position == -1:

    #         y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
    #         y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
    #         # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
    #         if df.loc[i, 'high'] >= short_initial_sl:
    #             tradebook_short.loc[
    #                 (tradebook_short['Trade No.'] == trade_number),
    #                 ['Exit_Date', 'Exit_Price', 'Exit Remark']
    #             ] = [df.loc[i, 'datetime'] , short_initial_sl , 'Initial SL Hit']
    #             short_position = 0
    #             can_add_short = False
    #             trade_number += 1
    #             lock_initial_sl_2 = False
    #             first_sl_2 = 0
            
    #         elif df.loc[i, 'close'] >= short_trailing_stop:
    #             tradebook_short.loc[
    #                 (tradebook_short['Trade No.'] == trade_number),
    #                 ['Exit_Date', 'Exit_Price', 'Exit Remark']
    #             ] = [df.loc[i, 'datetime'] , df.loc[i, 'close'] , 'Trailing SL Hit']
    #             short_position = 0
    #             can_add_short = False
    #             trade_number += 1
    #             lock_initial_sl_2 = False
    #             first_sl_2 = 0

    #         elif (df.loc[i, 'high'] >= y_days_high) and not can_add_short:
    #             # Previous Y Candle High Breached, confirmation found. Add position on low break
    #             can_add_short = True

    #         if can_add_short and short_position:
    #             if df.loc[i, 'low'] <= y_days_low:
    #                 tradebook_short = pd.concat([tradebook_short, pd.DataFrame([{
    #                     'Trade No.': trade_number,
    #                     'Entry_Date': df.loc[i, 'datetime'],
    #                     'Exit_Date': None,
    #                     'Trade_Type': 'Add_Short',
    #                     'Entry_Price': y_days_low,
    #                     'Initial SL': first_sl_2,
    #                     'Exit_Price': None,
    #                     # 'Profit/Loss': None  # Filled when exited
    #                 }])], ignore_index=True)
    #                 can_add_short = False
    
    tradebook = pd.concat([tradebook_long, tradebook_short], ignore_index=True)
    return tradebook


In [58]:
# def backtest(df, y_days):
#     # Variables for long trades
#     long_position = 0  # 0 = no position, 1 = long
#     long_entry_price = 0
#     long_entry_date = None
#     long_trades = []
#     tradebook = pd.DataFrame()
#     tradebook_long = pd.DataFrame()
#     tradebook_short = pd.DataFrame()
#     long_trailing_stop = None
#     # Variables for short trades
#     short_position = 0  # 0 = no position, -1 = short
#     short_entry_price = 0
#     short_entry_date = None
#     short_trades = []
#     short_trailing_stop = None

#     can_add_long = False
#     can_add_short = False

#     trade_number = 0

#     for i in range(len(df)):
#         # Entry signals for long trades
#         long_trailing_stop = df.loc[i, 'EMA_20']
#         short_trailing_stop = df.loc[i, 'EMA_20']
        
#         if df.loc[i, 'Buy_Signal'] and long_position == 0:
#             long_position = 1
#             long_entry_price = df.loc[i, 'close']
#             long_entry_date = df.loc[i, 'datetime']
#             long_trailing_stop = df.loc[i, 'EMA_20']
#             long_initial_sl = df.loc[i, 'X_Low']

#             tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
#                 'Trade No.': trade_number,
#                 'Entry_Date': df.loc[i, 'datetime'],
#                 'Exit_Date': None,
#                 'Trade_Type': 'Long',
#                 'Entry_Price': df.loc[i, 'close'],
#                 'Initial SL': long_initial_sl,
#                 'Exit_Price': None,
#                 # 'Profit/Loss': None  # Filled when exited
#             }])], ignore_index=True)

#         # Exit or add to long position
#         if long_position == 1:
            
#             y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
#             y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
#             # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
#             if df.loc[i, 'low'] <= long_initial_sl:
#                 tradebook_long.loc[
#                     (tradebook_long['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] , long_initial_sl]
#                 long_position = 0
#                 can_add_long = False
#                 trade_number += 1
            
#             elif df.loc[i, 'close'] <= long_trailing_stop:
#                 tradebook_long.loc[
#                     (tradebook_long['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] , df.loc[i, 'close']]
#                 long_position = 0
#                 can_add_long = False
#                 trade_number += 1

#             elif (df.loc[i, 'low'] <= y_days_low) and not can_add_long:
#                 # Previous Y Candle Low Breached, confirmation found. Add position on high break
#                 can_add_long = True

#             if can_add_long and long_position:
#                 if df.loc[i, 'high'] >= y_days_high:
#                     tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
#                         'Trade No.': trade_number,
#                         'Entry_Date': df.loc[i, 'datetime'],
#                         'Exit_Date': None,
#                         'Trade_Type': 'Add_Long',
#                         'Entry_Price': y_days_high,
#                         'Initial SL': long_initial_sl,
#                         'Exit_Price': None,
#                         # 'Profit/Loss': None  # Filled when exited
#                     }])], ignore_index=True)
#                     can_add_long = False

#     for i in range(len(df)):
        
#         # Entry signals for long trades
#         long_trailing_stop = df.loc[i, 'EMA_20']
#         short_trailing_stop = df.loc[i, 'EMA_20']

#         # Entry signals for short trades
#         if df.loc[i, 'Sell_Signal'] and short_position == 0:
#             short_position = -1
#             short_entry_price = df.loc[i, 'close']
#             short_entry_date = df.loc[i, 'datetime']
#             short_trailing_stop = df.loc[i, 'EMA_20']
#             short_initial_sl = df.loc[i, 'X_High']

#             tradebook_short = pd.concat([tradebook_short, pd.DataFrame([{
#                 'Trade No.': trade_number,
#                 'Entry_Date': df.loc[i, 'datetime'],
#                 'Exit_Date': None,
#                 'Trade_Type': 'Short',
#                 'Entry_Price': df.loc[i, 'close'],
#                 'Initial SL': short_initial_sl,
#                 'Exit_Price': None,
#                 # 'Profit/Loss': None  # Filled when exited
#             }])], ignore_index=True)

#         if short_position == -1:
            
#             y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
#             y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
#             # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
#             if df.loc[i, 'high'] >= short_initial_sl:
#                 tradebook_short.loc[
#                     (tradebook_short['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] ,short_initial_sl]
#                 short_position = 0
#                 can_add_short = False
#                 trade_number += 1
            
#             elif df.loc[i, 'close'] >= short_trailing_stop:
#                 tradebook_short.loc[
#                     (tradebook_short['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] , df.loc[i, 'close']]
#                 short_position = 0
#                 can_add_short = False
#                 trade_number += 1

#             elif (df.loc[i, 'high'] >= y_days_high) and not can_add_short:
#                 # Previous Y Candle High Breached, confirmation found. Add position on low break
#                 can_add_short = True

#             if can_add_short and short_position:
#                 if df.loc[i, 'low'] <= y_days_low:
#                     tradebook_short = pd.concat([tradebook_short, pd.DataFrame([{
#                         'Trade No.': trade_number,
#                         'Entry_Date': df.loc[i, 'datetime'],
#                         'Exit_Date': None,
#                         'Trade_Type': 'Add_Short',
#                         'Entry_Price': y_days_low,
#                         'Initial SL': short_initial_sl,
#                         'Exit_Price': None,
#                         # 'Profit/Loss': None  # Filled when exited
#                     }])], ignore_index=True)
#                     can_add_short = False
    
#     tradebook = pd.concat([tradebook_long, tradebook_short], ignore_index=True)
#     return tradebook


In [59]:
ema_length = 55
x_days = 28
y_days = 7
pct = 1.05
RPT = 5
df = bnf_1min
df = calculate_signals(df, ema_length=ema_length, x_days=x_days, pct=pct)
tb = backtest(df, y_days=y_days)
tb = tb.sort_values(by='Entry_Date')
variation = f'EMA: {ema_length}, X: {x_days}, Y: {y_days}, PCT: {pct}%, RPT: {RPT}%'

/tmp/ipykernel_30393/1354707490.py:75: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'MA Cross Reversal' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


In [60]:
import numpy as np
portfolio = 10000000
tb['PnL'] = np.where(
    tb['Trade_Type'].isin(['Long', 'Add_Long']),  # For Long and Add_Long
    tb['Exit_Price'] - tb['Entry_Price'],
    np.where(
        tb['Trade_Type'].isin(['Short', 'Add_Short']),  # For Short and Add_Short
        tb['Entry_Price'] - tb['Exit_Price'],
        0  # Default case if Trade_Type is something unexpected
    )
)

tb['Slippage'] = SLIPPAGE_ * (tb['Entry_Price'] + tb['Exit_Price'])
tb['PnL w cs'] = tb['PnL'] - tb['Slippage']
tb['Qty'] = abs(RPT / 100 * portfolio / (tb['Entry_Price'] - tb['Initial SL']))
tb['Profit_Loss INR'] = tb['Qty'] * tb['PnL w cs']
tb['ROI%'] = tb['Profit_Loss INR'] * 100 / portfolio

In [61]:
tb['Cumulative_PnL'] = tb['PnL w cs'].cumsum()

In [62]:
# tb.to_csv('GOLD TRADE CHECK JJPP.csv', index=False)

In [63]:
tb = tb.reset_index()
# tb

In [64]:
import matplotlib.pyplot as plt

def plot_cumulative_pnl(tb, title="Cumulative PnL Over Time"):
    if 'Cumulative_PnL' not in tb.columns:
        raise ValueError("The DataFrame must contain a 'Cumulative_PnL' column.")

    # Plot cumulative PnL
    plt.figure(figsize=(12, 3))
    plt.plot(tb.index, tb['Cumulative_PnL'], label='Cumulative PnL', color='blue', linewidth=2)
    plt.title(title, fontsize=8)
    plt.xlabel('Trade Index', fontsize=6)
    plt.ylabel('Cumulative PnL', fontsize=6)
    plt.legend(fontsize=6)
    plt.grid(alpha=0.5)
    plt.show()


In [65]:
tb_long_only = tb[(tb['Trade_Type'] == 'Long') | (tb['Trade_Type'] == 'Add_Long')].copy()
tb_long_only['Cumulative_PnL'] = tb_long_only['PnL w cs'].cumsum()
tb_short_only = tb[(tb['Trade_Type'] == 'Short') | (tb['Trade_Type'] == 'Add_Short')].copy()
tb_short_only['Cumulative_PnL'] = tb_short_only['PnL w cs'].cumsum()

In [66]:
# tb2 = tb.groupby('Trade No.', as_index=False)['PnL w cs'].sum()
# tb2['Cumulative_PnL'] = tb2['PnL w cs'].cumsum()

In [67]:
# plot_cumulative_pnl(tb_long_only)
# print(tb_long_only['PnL w cs'].sum())

In [68]:
# plot_cumulative_pnl(tb_short_only)
# print(tb_short_only['PnL w cs'].sum())

In [69]:
# plot_cumulative_pnl(tb)
# print(tb['PnL w cs'].sum())

In [70]:
# plot_cumulative_pnl(tb2)
# print(tb2['PnL w cs'].sum())

In [71]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2015, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2015, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    
    # print(f'{overall_total_roi} , {overall_max_drawdown} , {overall_roi_dd_ratio}')
    
    return {overall_roi_dd_ratio: stats_df8}

In [72]:
tb_long_only['Entry_Date'] = pd.to_datetime(tb_long_only['Entry_Date'])
tb_long_only['Trade Year'] = tb_long_only['Entry_Date'].dt.year
tb_long_only['ROI% w cs'] = tb_long_only['ROI%']

In [73]:
stats = generate_stats(tb_long_only, variation)
lol = pd.DataFrame()
for x, y in stats.items():
    lol = pd.DataFrame(y)

lol

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2015,-31.3579,27,25.9259,3.8960,-2.9315,-34.4549,-0.9101,"EMA: 55, X: 28, Y: 7, PCT: 1.05%, RPT: 5%"
2016,86.6569,30,43.3333,10.1305,-2.6494,-23.7365,3.6508,"EMA: 55, X: 28, Y: 7, PCT: 1.05%, RPT: 5%"
2017,-4.2205,30,33.3333,4.6431,-2.5326,-25.8387,-0.1633,"EMA: 55, X: 28, Y: 7, PCT: 1.05%, RPT: 5%"
2018,-17.0873,32,34.3750,2.9012,-2.3333,-31.0197,-0.5509,"EMA: 55, X: 28, Y: 7, PCT: 1.05%, RPT: 5%"
2019,24.1537,29,37.9310,7.5659,-3.2817,-31.4530,0.7679,"EMA: 55, X: 28, Y: 7, PCT: 1.05%, RPT: 5%"
2020,-1.7674,36,25.0000,8.3492,-2.8485,-46.4967,-0.0380,"EMA: 55, X: 28, Y: 7, PCT: 1.05%, RPT: 5%"
2021,-25.4742,34,29.4118,3.7346,-2.6175,-37.7369,-0.6750,"EMA: 55, X: 28, Y: 7, PCT: 1.05%, RPT: 5%"
2022,44.6909,32,40.6250,7.7928,-2.9798,-39.0813,1.1435,"EMA: 55, X: 28, Y: 7, PCT: 1.05%, RPT: 5%"
2023,26.7247,33,45.4545,3.2574,-1.2298,-10.4833,2.5493,"EMA: 55, X: 28, Y: 7, PCT: 1.05%, RPT: 5%"
2024,77.4839,31,41.9355,9.2501,-2.3760,-20.5401,3.7723,"EMA: 55, X: 28, Y: 7, PCT: 1.05%, RPT: 5%"


In [74]:
tb_long_only

,index,Trade No.,Entry_Date,Exit_Date,Trade_Type,Entry_Price,Initial SL,Exit_Price,Exit Remark,PnL,Slippage,PnL w cs,Qty,Profit_Loss INR,ROI%,Cumulative_PnL,Trade Year,ROI% w cs
0,0,0,2015-03-20 17:00:00+05:30,2015-04-01 09:00:00+05:30,Long,26108.0000,25716.3800,26320,MA Cross Reversal,212.0000,52.4280,159.5720,1276.7479,203733.2108,2.0373,159.5720,2015,2.0373
1,1,1,2015-04-01 17:00:00+05:30,2015-04-10 17:00:00+05:30,Long,26620.0000,26220.7000,26758,MA Cross Reversal,138.0000,53.3780,84.6220,1252.1913,105962.9351,1.0596,244.1940,2015,1.0596
2,2,2,2015-04-13 17:00:00+05:30,2015-04-15 17:00:00+05:30,Long,26669.0000,26268.9650,26586,MA Cross Reversal,-83.0000,53.2550,-136.2550,1249.8906,-170303.8484,-1.7030,107.9390,2015,-1.7030
3,3,3,2015-04-17 13:00:00+05:30,2015-04-24 17:00:00+05:30,Long,26745.0000,26343.8250,26674,MA Cross Reversal,-71.0000,53.4190,-124.4190,1246.3389,-155068.2371,-1.5507,-16.4800,2015,-1.5507
4,4,3,2015-04-24 13:00:00+05:30,2015-04-24 17:00:00+05:30,Add_Long,26929.0000,26343.8250,26674,MA Cross Reversal,-255.0000,53.6030,-308.6030,854.4453,-263684.3679,-2.6368,-325.0830,2015,-2.6368
5,5,4,2015-04-24 21:00:00+05:30,2015-05-04 21:00:00+05:30,Long,26720.0000,26319.2000,26868,MA Cross Reversal,148.0000,53.5880,94.4120,1247.5050,117779.4411,1.1778,-230.6710,2015,1.1778
6,6,4,2015-04-27 17:00:00+05:30,2015-05-04 21:00:00+05:30,Add_Long,26959.0000,26319.2000,26868,MA Cross Reversal,-91.0000,53.8270,-144.8270,781.4942,-113181.4630,-1.1318,-375.4980,2015,-1.1318
7,7,5,2015-05-07 13:00:00+05:30,2015-05-11 17:00:00+05:30,Long,26967.0000,26562.4950,26802,MA Cross Reversal,-165.0000,53.7690,-218.7690,1236.0787,-270415.6933,-2.7042,-594.2670,2015,-2.7042
8,8,6,2015-05-12 17:00:00+05:30,2015-05-21 17:00:00+05:30,Long,27145.0000,26737.8250,27141,MA Cross Reversal,-4.0000,54.2860,-58.2860,1227.9732,-71573.6477,-0.7157,-652.5530,2015,-0.7157
9,9,7,2015-06-03 13:00:00+05:30,2015-06-05 13:00:00+05:30,Long,27140.0000,26732.9000,26732.9000,Initial SL Hit,-407.1000,53.8729,-460.9729,1228.1995,-566166.6667,-5.6617,-1113.5259,2015,-5.6617


In [17]:
# tb_long_only.to_csv('GOLD Range JJPP.csv', index=False)

In [18]:
tb_long_only_1 = tb_long_only[tb_long_only['Trade_Type'] == 'Add_Long']
tb_long_only_2 = tb_long_only[tb_long_only['Trade_Type'] == 'Long']

NameError: name 'tb_long_only' is not defined

In [19]:
tb_long_only_1['ROI%'].sum() , tb_long_only_2['ROI%'].sum()

NameError: name 'tb_long_only_1' is not defined

In [ ]:
# ema_length = 55
# x_days = 28
# y_days = 7
# pct = 1.05
# RPT = 5

pct_range = [0.5, 0.75, 0.9, 1.05, 1.25, 1.4]

for ema_length in range(25, 81, 5):
    for x_days in range(4, 37, 4):
        for y_days in range(3, 19, 3):
            for pct in pct_range:
                RPT = 5
                variation = f'EMA:{ema_length}, X:{x_days}, Y:{y_days}, PCT:{pct}, RPT:{5}'
                print(variation)
                df = bnf_1min
                df = calculate_signals(df, ema_length=ema_length, x_days=x_days, pct=pct)
                tb = backtest(df, y_days=y_days)
                if len(tb) > 0:
                    tb = tb.sort_values(by='Entry_Date')
                    portfolio = 10000000
                    tb['PnL'] = np.where(
                        tb['Trade_Type'].isin(['Long', 'Add_Long']),  # For Long and Add_Long
                        tb['Exit_Price'] - tb['Entry_Price'],
                        np.where(
                            tb['Trade_Type'].isin(['Short', 'Add_Short']),  # For Short and Add_Short
                            tb['Entry_Price'] - tb['Exit_Price'],
                            0  # Default case if Trade_Type is something unexpected
                        )
                    )
                    
                    tb['Slippage'] = SLIPPAGE_ * (tb['Entry_Price'] + tb['Exit_Price'])
                    tb['PnL w cs'] = tb['PnL'] - tb['Slippage']
                    tb['Qty'] = abs(RPT / 100 * portfolio / (tb['Entry_Price'] - tb['Initial SL']))
                    tb['Profit_Loss INR'] = tb['Qty'] * tb['PnL w cs']
                    tb['ROI%'] = tb['Profit_Loss INR'] * 100 / portfolio
                    tb['Entry_Date'] = pd.to_datetime(tb['Entry_Date'])
                    tb['Trade Year'] = tb['Entry_Date'].dt.year
                    tb_long_only = tb[(tb['Trade_Type'] == 'Long') | (tb['Trade_Type'] == 'Add_Long')].copy()
                    tb_long_only['Cumulative_PnL'] = tb_long_only['PnL w cs'].cumsum()
                    tb_short_only = tb[(tb['Trade_Type'] == 'Short') | (tb['Trade_Type'] == 'Add_Short')].copy()
                    tb_short_only['Cumulative_PnL'] = tb_short_only['PnL w cs'].cumsum()

                    stats1 = generate_stats(tb_long_only, variation)
                    for x, y in stats1.items():
                        if x > 2:
                            print('Long Only')
                            print(pd.DataFrame(y).to_string())
                        break
                    stats2 = generate_stats(tb_short_only, variation)
                    for x, y in stats2.items():
                        if x > 2:
                            print('Short Only')
                            print(pd.DataFrame(y).to_string())
                        break

EMA:25, X:4, Y:3, PCT:0.5, RPT:5


/tmp/ipykernel_4593/453456106.py:76: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[
/tmp/ipykernel_4593/453456106.py:152: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_short.loc[


Long Only
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                         Variation
2015     -10.4907           37  27.0270                9.5618             -3.9300     -53.4513      -0.1963  EMA:25, X:4, Y:3, PCT:0.5, RPT:5
2016     110.7251           49  38.7755               10.2685             -2.8126     -45.3811       2.4399  EMA:25, X:4, Y:3, PCT:0.5, RPT:5
2017     -24.7094           50  24.0000                5.3332             -2.3344     -58.6098      -0.4216  EMA:25, X:4, Y:3, PCT:0.5, RPT:5
2018     -34.4612           41  34.1463                3.8950             -3.2960     -62.1415      -0.5546  EMA:25, X:4, Y:3, PCT:0.5, RPT:5
2019      15.7352           53  30.1887                8.4490             -3.2283     -63.4594       0.2480  EMA:25, X:4, Y:3, PCT:0.5, RPT:5
2020      52.1499           56  35.7143                7.6980             -2.8280     -48.1340       1.0834  EMA:25, X:4, Y:3, PCT:0.5, RP

/tmp/ipykernel_4593/453456106.py:76: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[
/tmp/ipykernel_4593/453456106.py:152: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_short.loc[


Long Only
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                          Variation
2015     -33.3079           22  18.1818                9.3994             -3.9392     -39.1528      -0.8507  EMA:25, X:4, Y:3, PCT:0.75, RPT:5
2016      89.5199           41  39.0244                9.9028             -2.7570     -37.4115       2.3928  EMA:25, X:4, Y:3, PCT:0.75, RPT:5
2017     -23.8991           32  25.0000                3.8688             -2.2854     -45.7482      -0.5224  EMA:25, X:4, Y:3, PCT:0.75, RPT:5
2018      -5.2815           24  41.6667                4.3309             -3.4707     -34.6120      -0.1526  EMA:25, X:4, Y:3, PCT:0.75, RPT:5
2019      10.9014           33  36.3636                6.7004             -3.3097     -43.0486       0.2532  EMA:25, X:4, Y:3, PCT:0.75, RPT:5
2020      32.5295           41  34.1463                6.9959             -2.4227     -33.9253       0.9589  EMA:25, X:4, Y:3, PCT:0

...

/tmp/ipykernel_4593/453456106.py:76: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[
/tmp/ipykernel_4593/453456106.py:152: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_short.loc[


EMA:30, X:16, Y:3, PCT:0.9, RPT:5


/tmp/ipykernel_4593/453456106.py:76: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[
/tmp/ipykernel_4593/453456106.py:152: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_short.loc[


Long Only
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio                          Variation
2015      -8.8652           21  33.3333                4.9113             -3.0889     -25.7504      -0.3443  EMA:30, X:16, Y:3, PCT:0.9, RPT:5
2016      98.9504           44  38.6364                9.4734             -2.2999     -34.2506       2.8890  EMA:30, X:16, Y:3, PCT:0.9, RPT:5
2017     -26.1436           39  25.6410                4.5411             -2.4674     -62.1024      -0.4210  EMA:30, X:16, Y:3, PCT:0.9, RPT:5
2018      14.8884           36  47.2222                3.3581             -2.2211     -26.3250       0.5656  EMA:30, X:16, Y:3, PCT:0.9, RPT:5
2019      51.8225           40  37.5000                6.5763             -1.8728     -29.1141       1.7800  EMA:30, X:16, Y:3, PCT:0.9, RPT:5
2020      -4.8994           48  29.1667                5.3292             -2.3385     -47.1509      -0.1039  EMA:30, X:16, Y:3, PCT:

/tmp/ipykernel_4593/453456106.py:76: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[
/tmp/ipykernel_4593/453456106.py:152: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_short.loc[
